# Cenpy Tutorial 
Author: Zach Schira

## About 
There are several useful online sources for accessing census data provided both by the US census Bureau ([American Factfinder](http://factfinder.census.gov)), and outside sources. These sources, however, are not conducive to large scale data aquisition and analysis. The [Cenpy](https://pypi.python.org/pypi/cenpy/0.9.1) python package allows for programmitic access of this data through the [Census Bureau's API](http://www.census.gov/data/developers/data-sets.html).

This tutorial outlines the use of the Cenpy package to search for, and acquire specific census data. Cenpy saves this data as a [Pandas](http://pandas.pydata.org/) dataframe. These dataframes allow for easy access and analysis of data within python. For easy visualization of this data look into the [GeoPandas](http://geopandas.org/) package. This package builds on the base Pandas package to add tools for geospatial data analysis.

## Objectives
- Install Cenpy package
- Search for desired census data
- Download and store data

## Dependencies 
The Cenpy package depends on pandas and [requests](http://docs.python-requests.org/en/master/). Ensure that python and pip are already properly installed then use the following commands to install cenpy.

In [ ]:
!pip install pandas
!pip install requests
!pip install cenpy

In [1]:
import pandas as pd
import cenpy as cen

## Finding Data
The cenpy explorer module allows you to view all of the available [Census API's](http://www.census.gov/data/developers/data-sets.html). Passing the name of a specific API to `explorer.explain()` will give a description of the data available.

For this example, we will use the 2012 [American Community Service](https://www.census.gov/programs-surveys/acs/) 1 year data (`2012acs1`).

In [2]:
datasets = list(cen.explorer.available(verbose=True).items())
print('Number of datasets: ', len(datasets))
#for i in range(0,len(datasets)):
#    print(datasets[i][0],': ',datasets[i][1])
dataset = '2012acs1'
cen.explorer.explain(dataset)

Number of datasets:  125


{'2012 American Community Survey: 1-Year Estimates': "The American Community Survey (ACS) is a nationwide survey designed to provide communities a fresh look at how they are changing. The ACS replaced the decennial census long form in 2010 and thereafter by collecting long form type information throughout the decade rather than only once every 10 years.  Questionnaires are mailed to a sample of addresses to obtain information about households -- that is, about each person and the housing unit itself.  The American Community Survey produces demographic, social, housing and economic estimates in the form of 1-year, 3-year and 5-year estimates based on population thresholds. The strength of the ACS is in estimating population and housing characteristics. It produces estimates for small areas, including census tracts and population subgroups.  Although the ACS produces population, demographic and housing unit estimates,it is the Census Bureau's Population Estimates Program that produces an

The base module allows you to establish a connection with the desired api that will be used later to acquire data

In [3]:
con = cen.base.Connection(dataset)
con

Connection to 2012 American Community Survey: 1-Year Estimates (ID: http://api.census.gov/data/id/2012acs1)

## Getting Data
### Geography
Cenpy uses [fips codes](https://www.census.gov/geo/reference/codes/cou.html) to specify the geographical extent of the data to be downloaded.

In [4]:
con.geographies.keys()

dict_keys(['fips'])

In [5]:
con.geographies['fips']

,geoLevelId,name,optionalWithWCFor,requires
0,500,congressional district,state,[state]
1,060,county subdivision,NaN,"[state, county]"
2,795,public use microdata area,NaN,[state]
3,320,metropolitan statistical area/micropolitan sta...,NaN,[state]
4,310,metropolitan statistical area/micropolitan sta...,NaN,NaN
5,160,place,state,[state]
6,050,county,state,[state]
7,NaN,combined statistical area,NaN,NaN
8,NaN,combined new england city and town area,NaN,NaN
9,NaN,new england city and town area,NaN,NaN


`geo_unit` and `geo_filter` are both necessary arguments for the `query()` function. `geo_unit` specifies the scale at which data should be taken. `geo_filter` then creates a filter to ensure too much data is not downloaded. The following example will download data from all counties in Colorado.

In [6]:
g_unit = 'county:*'
g_filter = {'state':'8'}

### Variables
The other argument taken by `query()` is cols. This is a list of columns taken from the variables of the API. These variables can be displayed using the `variables` function, however, due to the number of variables it is easier to use the [Social Explorer](https://www.socialexplorer.com/) site to find data you are interested in.

In [7]:
var = con.variables
print('Number of variables in', dataset, ':', len(var))

Number of variables in 2012acs1 : 68401


Related columns of data will always start with the same base prefix, so Cenpy has an included function, `varslike`, that will create a list of column names that match the input pattern. It is also useful to add on the `NAME` and `GEOID` columns, as these will provide the name and geographic id of all data. In this example, we will use the [B01001A](https://www.socialexplorer.com/data/ACS2013/metadata/?ds=ACS13&table=B01001A), which gives data for sex by age within the desired geography. The identifier at the end corresponds to males or females of different age groups.

In [8]:
cols = con.varslike('B01001A_')
cols.extend(['NAME', 'GEOID'])
cols

['B01001A_001E',
 'B01001A_001M',
 'B01001A_002E',
 'B01001A_002M',
 'B01001A_003E',
 'B01001A_003M',
 'B01001A_004E',
 'B01001A_004M',
 'B01001A_005E',
 'B01001A_005M',
 'B01001A_006E',
 'B01001A_006M',
 'B01001A_007E',
 'B01001A_007M',
 'B01001A_008E',
 'B01001A_008M',
 'B01001A_009E',
 'B01001A_009M',
 'B01001A_010E',
 'B01001A_010M',
 'B01001A_011E',
 'B01001A_011M',
 'B01001A_012E',
 'B01001A_012M',
 'B01001A_013E',
 'B01001A_013M',
 'B01001A_014E',
 'B01001A_014M',
 'B01001A_015E',
 'B01001A_015M',
 'B01001A_016E',
 'B01001A_016M',
 'B01001A_017E',
 'B01001A_017M',
 'B01001A_018E',
 'B01001A_018M',
 'B01001A_019E',
 'B01001A_019M',
 'B01001A_020E',
 'B01001A_020M',
 'B01001A_021E',
 'B01001A_021M',
 'B01001A_022E',
 'B01001A_022M',
 'B01001A_023E',
 'B01001A_023M',
 'B01001A_024E',
 'B01001A_024M',
 'B01001A_025E',
 'B01001A_025M',
 'B01001A_026E',
 'B01001A_026M',
 'B01001A_027E',
 'B01001A_027M',
 'B01001A_028E',
 'B01001A_028M',
 'B01001A_029E',
 'B01001A_029M',
 'B01001A_030E

With the three necessary arguments, data can be downloaded and saved as a panda dataframe.

In [9]:
data = con.query(cols, geo_unit=g_unit, geo_filter=g_filter)

C:\Users\Zach\Anaconda3\lib\site-packages\cenpy\remote.py:167: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df[cols] = df[cols].convert_objects(convert_numeric=convert_numeric)


It is useful to replace the default index with the data from the `NAME` or `GEOID` column, as these will give a more useful description of the data.

In [10]:
data.index = data.NAME
#data.index = data.GEOID
data[['B01001A_001E']]

,B01001A_001E
NAME,
"Adams County, Colorado",380017
"Arapahoe County, Colorado",445459
"Boulder County, Colorado",270200
"Denver County, Colorado",479745
"Douglas County, Colorado",271346
"El Paso County, Colorado",527453
"Jefferson County, Colorado",495398
"Larimer County, Colorado",281231
"Mesa County, Colorado",136439


### Tiger
The Census [TIGER API](https://www.census.gov/geo/maps-data/data/tiger.html) provides geomotries for desired geographic regions.

In [11]:
cen.tiger.available()

[{'name': 'AIANNHA', 'type': 'MapServer'},
 {'name': 'CBSA', 'type': 'MapServer'},
 {'name': 'Hydro_LargeScale', 'type': 'MapServer'},
 {'name': 'Hydro', 'type': 'MapServer'},
 {'name': 'Labels', 'type': 'MapServer'},
 {'name': 'Legislative', 'type': 'MapServer'},
 {'name': 'Places_CouSub_ConCity_SubMCD', 'type': 'MapServer'},
 {'name': 'PUMA_TAD_TAZ_UGA_ZCTA', 'type': 'MapServer'},
 {'name': 'Region_Division', 'type': 'MapServer'},
 {'name': 'School', 'type': 'MapServer'},
 {'name': 'Special_Land_Use_Areas', 'type': 'MapServer'},
 {'name': 'State_County', 'type': 'MapServer'},
 {'name': 'tigerWMS_ACS2013', 'type': 'MapServer'},
 {'name': 'tigerWMS_ACS2014', 'type': 'MapServer'},
 {'name': 'tigerWMS_ACS2015', 'type': 'MapServer'},
 {'name': 'tigerWMS_Census2010', 'type': 'MapServer'},
 {'name': 'tigerWMS_Current', 'type': 'MapServer'},
 {'name': 'tigerWMS_Econ2012', 'type': 'MapServer'},
 {'name': 'tigerWMS_PhysicalFeatures', 'type': 'MapServer'},
 {'name': 'Tracts_Blocks', 'type': 'Ma

First, you must establish a connection to the TIGER API, then you can display the avaialable layers. No Tiger data is available for ACS 2012, so we will use the ACS 2013 for the sake of example, but ideally you will be able to find corresponding Tiger data.

In [12]:
con.set_mapservice('tigerWMS_ACS2013')
con.mapservice.layers

{0: (ESRILayer) 2010 Census Public Use Microdata Areas,
 1: (ESRILayer) 2010 Census Public Use Microdata Areas Labels,
 2: (ESRILayer) 2010 Census ZIP Code Tabulation Areas,
 3: (ESRILayer) 2010 Census ZIP Code Tabulation Areas Labels,
 4: (ESRILayer) Tribal Census Tracts,
 5: (ESRILayer) Tribal Census Tracts Labels,
 6: (ESRILayer) Tribal Block Groups,
 7: (ESRILayer) Tribal Block Groups Labels,
 8: (ESRILayer) Census Tracts,
 9: (ESRILayer) Census Tracts Labels,
 10: (ESRILayer) Census Block Groups,
 11: (ESRILayer) Census Block Groups Labels,
 12: (ESRILayer) Unified School Districts,
 13: (ESRILayer) Unified School Districts Labels,
 14: (ESRILayer) Secondary School Districts,
 15: (ESRILayer) Secondary School Districts Labels,
 16: (ESRILayer) Elementary School Districts,
 17: (ESRILayer) Elementary School Districts Labels,
 18: (ESRILayer) Estates,
 19: (ESRILayer) Estates Labels,
 20: (ESRILayer) County Subdivisions,
 21: (ESRILayer) County Subdivisions Labels,
 22: (ESRILayer) 

The data retrieved earlier was at the county level, so we will use layer 84.

In [13]:
con.mapservice.layers[84]

(ESRILayer) Counties

Now, using the tiger connection, `query()` can be called providing the layer and the geographic location as the two arguments.

In [14]:
geodata = con.mapservice.query(layer=84, where='STATE=8')

This data can now be merged with the original data to create one panda dataframe containing all of the relevant data

In [15]:
newdata = pd.merge(data, geodata, left_on='county', right_on='COUNTY')
newdata

,B01001A_001E,B01001A_001M,B01001A_002E,B01001A_002M,B01001A_003E,B01001A_003M,B01001A_004E,B01001A_004M,B01001A_005E,B01001A_005M,...,GEOID_y,INTPTLAT,INTPTLON,LSADC,MTFCC,NAME_y,OBJECTID,OID,STATE,geometry
0,380017,6171,190078,3378,14551,903,16003,1466,13330,1343,...,08001,+39.8743252,-104.3318718,06,G4020,Adams County,1226,27553700234319,08,<pysal.cg.shapes.Polygon object at 0x000002522...
1,445459,5621,218157,3358,13536,742,15491,1398,13177,1520,...,08005,+39.6445537,-104.3317065,06,G4020,Arapahoe County,2980,27553703789414,08,<pysal.cg.shapes.Polygon object at 0x000002522...
2,270200,3059,136575,1499,6977,321,6985,715,8421,847,...,08013,+40.0949699,-105.3976911,06,G4020,Boulder County,512,27553701435070,08,<pysal.cg.shapes.Polygon object at 0x000002522...
3,479745,6561,241272,3883,17073,954,13994,1779,10457,1465,...,08031,+39.7618502,-104.8811054,06,G4020,Denver County,529,27553700234321,08,<pysal.cg.shapes.Polygon object at 0x000002522...
4,271346,1601,135162,1093,9516,354,12390,1074,11748,1049,...,08035,+39.3254140,-104.9259871,06,G4020,Douglas County,2762,27553711656416,08,<pysal.cg.shapes.Polygon object at 0x000002522...
5,527453,4503,264361,2801,17916,599,18957,1465,17433,1477,...,08041,+38.8273831,-104.5274718,06,G4020,El Paso County,2878,27553704502958,08,<pysal.cg.shapes.Polygon object at 0x000002522...
6,495398,3342,246793,1492,12941,361,14763,1470,15163,1344,...,08059,+39.5795106,-105.2454623,06,G4020,Jefferson County,540,27553702223972,08,<pysal.cg.shapes.Polygon object at 0x000002522...
7,281231,3020,139420,1665,7459,454,8569,802,7141,1004,...,08069,+40.6580933,-105.4867638,06,G4020,Larimer County,1691,27553352525178,08,<pysal.cg.shapes.Polygon object at 0x000002522...
8,136439,1926,67852,1173,4594,780,4456,918,4048,839,...,08077,+39.0194206,-108.4618935,06,G4020,Mesa County,2925,27553356301770,08,<pysal.cg.shapes.Polygon object at 0x000002522...
9,128522,4049,62431,2107,3472,550,3488,750,4246,940,...,08101,+38.1706581,-104.4898924,06,G4020,Pueblo County,653,27553704502959,08,<pysal.cg.shapes.Polygon object at 0x000002522...
